# Comparison of spectrum between eNATL60 outputs and Alitka Satellite 

Requisites :
 - git clone https://github.com/auraoupa/gonzag_cloud
 - git clone https://github.com/brodeau/climporn
 
Assumes that time is encoded in datetime64 format in both datasets.

Looks for model months in the longest year of Altika serie

In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud/gonzag' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(GONZAG_DIR)
import gonzag as gz


## Params

For the model

In [3]:
model = 'GIGATL'
region = '1'
season = 'fma'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'zeta'
name_lsm_mod = 'salt' 
name_lat_mod = 'lat'
name_lon_mod = 'lon'
name_time_mod = 'ocean_time'
l_griddist = False
period_mod = ['2000-02-01','2000-04-30']

For altimetry data

In [4]:
name_sat= 'Altika'
tag_sat_catalog = 'al'
name_ssh_sat='sla_unfiltered'
name_time_sat='time'
period_sat = ['2014-02-01','2014-04-30'] #should cover the same period of a different year if needed

## Data

In [5]:
import xarray as xr
import intake
import gcsfs

gcs = gcsfs.GCSFileSystem(requester_pays=True)

cat = intake.open_catalog('../SWOT-AdAC/catalog.yaml')
dsmod = cat[model](region=region,depth='surf', season=season).to_dask()

In [6]:
dsmod

<xarray.Dataset>
Dimensions:     (eta_rho: 1489, eta_v: 1488, time: 2280, xi_rho: 1491, xi_u: 1490)
Dimensions without coordinates: eta_rho, eta_v, time, xi_rho, xi_u
Data variables: (12/13)
    angle       (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    f           (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    h           (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    lat         (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    lon         (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    ocean_time  (time) float32 dask.array<chunksize=(24,), meta=np.ndarray>
    ...          ...
    pn          (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    salt        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>
    temp        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>
    u           (time, eta_rho, xi_u) float32 dask.array<chunksize=(24, 1489, 1490), meta=np.ndarray>
    v           (time, eta_v, xi_rho) float32 dask.array<chunksize=(24, 1488, 1491), meta=np.ndarray>
    zeta        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>

In [7]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
dssat  = cat[tag_sat_catalog].to_dask()

In [8]:
dssat

<xarray.Dataset>
Dimensions:         (time: 34340107)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2013-03-14T05:44:50.445185 ... 2015...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
Attributes: (12/24)
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    ...                        ...
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Altika measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Altika Global Ocean Along track SSALTO/DUAC...

## Model and Satellite objects

In [9]:
ModelGrid = gz.ModGrid( dsmod, period_mod, name_lon_mod, name_lat_mod , name_time_mod, dsmod, name_lsm_mod, distorded_grid=False )



 *** what we use to define model land-sea mask:
    => "salt" in dataset 

 *** Skipping computation of angle distortion of the model grid! ("-D" option not invoked)...

 *** About model gridded (source) domain:
     * shape =  (1489, 1491)
     * horizontal resolution:  0.007546403  degrees or  0.8384808350680396  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [0:360] frame...
     * lon_min, lon_max =  279.15 298.04
     * lat_min, lat_max =  25.86 42.41
     * should we pay attention to possible STRONG local distorsion in the grid:  False
     * number of time records of interest for the interpolation to come:  2136
       ==> time record dates: 2000-02-01 to 2000-04-30, included



In [10]:
SatelliteTrack = gz.SatTrack( dssat, period_sat, name_time_sat, name_ssh_sat, domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in dataset ...

 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  29724
       ==> time record indices: 14883510 to 19033581, included

       separated in 154 tracks


## Process the tracks in parallel (results are saved in separate netcdf files)

In [11]:
if not os.path.exists('results_'+name_sat+'-'+name_mod+'-region'+region+'-'+season):
        os.makedirs('results_'+name_sat+'-'+name_mod+'-region'+region+'-'+season)

In [12]:
def process_one_track(track):
    tt = "{:02d}".format(track)
    Solution0 = gz.Model2SatTrack( ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat, track )
    c1     = 'Model SSH interpolated in space (' ; c2=') and time on satellite track'
    vvar   = [ 'latitude', 'longitude', name_ssh_mod+'_np'   , name_ssh_mod+'_bl' , name_ssh_sat          , 'distance'                            ]
    vunits = [ 'deg.N'   , 'deg.E'    ,          'm'         ,     'm'            ,    'm'                ,    'km'                               ]
    vlongN = [ 'Latitude', 'Longitude', c1+'nearest-point'+c2,  c1+'bilinear'+c2  , 'Input satellite data', 'Cumulated distance from first point' ]
    from gonzag.config import rmissval
    iw = gz.io.SaveTimeSeries( Solution0.time, \
                             nmp.array( [Solution0.lat, Solution0.lon, Solution0.ssh_mod_np,
                                         Solution0.ssh_mod, Solution0.ssh_sat, Solution0.distance] ), \
                             vvar, 'results_'+name_sat+'-'+name_mod+'/result_'+str(tt)+'.nc', time_units='',\
                             vunits=vunits, vlnm=vlongN, missing_val=rmissval )
    return Solution0


In [14]:
track=0

In [16]:
    tt = "{:02d}".format(track)


In [17]:
(MG, name_ssh_mod, ST, name_ssh_sat, one_track) = (ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat, track)

In [18]:
        from gonzag.io   import GetModel2DVar, GetSatSSH


In [20]:
import time ; # to report execution speed of certain parts of the code...
import pandas as pd
import numpy as np
from gonzag.config import ldebug, ivrb, nb_talk, l_plot_meshes, deg2km, rfactor, search_box_w_km, l_save_track_on_model_grid, l_plot_meshes, rmissval
from gonzag.utils  import *
from gonzag.bilin_mapping import BilinTrack


In [21]:
        jt1=ST.index_tracks[one_track][0]
        jt2=ST.index_tracks[one_track][1]

        (Nj,Ni) = MG.shape

        d_found_km = rfactor*MG.HResDeg*deg2km
        #print(' *** "found" distance criterion when searching for nearest point on model grid is ', d_found_km, ' km\n')

        # Size of the search zoom box:
        np_box_radius = SearchBoxSize( MG.HResDeg*deg2km, search_box_w_km )
        #print(' *** Will use zoom boxes of width of '+str(2*np_box_radius+1)+' points for 1st attempts of nearest-point location...\n')

        Nt = jt2 - jt1 ; # number of satellit observation point to work with here...

        if_talk = Nt//nb_talk


In [22]:
        BT = BilinTrack( ST.lat[jt1:jt2], ST.lon[jt1:jt2], MG.lat, MG.lon, src_grid_local_angle=MG.xangle, k_ew_per=MG.EWPer, rd_found_km=d_found_km, np_box_r=np_box_radius, freq_talk=if_talk )


In [24]:
jt=jt1

In [25]:
[jj,ji] = BT.NP[jt-jt1,:]

In [26]:
PlotMesh( (ST.lon[jt],ST.lat[jt]), MG.lat, MG.lon, BT.SM[jt-jt1,:,:], BT.WB[jt-jt1,:], \
                                  fig_name='mesh_jt'+'%5.5i'%(jt)+'.png' )

In [27]:
        vssh_m_np = nmp.zeros(Nt) ; vssh_m_np[:] = rmissval; # vector to store the model data interpolated in time and space (nearest-point) on the satellite track...
        vssh_m_bl = nmp.zeros(Nt) ; vssh_m_bl[:] = rmissval; # vector to store the model data interpolated in time and space (bilinear) on the satellite track...
        vdistance = nmp.zeros(Nt)

        # Time increment on the satellite time:
        ktm1   = 0   ; ktm2   = 0
        ktm1_o = -10 ; ktm2_o = -10

        #print('\n *** Starting space-time interpolation of model data onto the '+str(Nt)+' selected track points...')



In [28]:
            itt = ST.time[jt] ; # unix time


In [29]:
print(itt)

2014-02-02T10:58:18.622069000


In [30]:
            year_sat=pd.to_datetime(itt).year


In [31]:
print(year_sat)

2014


In [32]:
            date_model=pd.Series(MG.time)


In [33]:
print(date_model)

0      2000-02-01 00:28:48
1      2000-02-01 01:29:36
2      2000-02-01 02:29:20
3      2000-02-01 03:29:04
4      2000-02-01 04:28:48
               ...        
2131   2000-04-29 19:29:04
2132   2000-04-29 20:28:48
2133   2000-04-29 21:29:36
2134   2000-04-29 22:29:20
2135   2000-04-29 23:29:04
Length: 2136, dtype: datetime64[ns]


In [34]:
date_model_satyear=date_model.apply(lambda dt: dt.replace(year=year_sat))

ValueError: day is out of range for month

In [35]:
dd=date_model[0]

In [36]:
print(dd)

2000-02-01 00:28:48


In [37]:
dd.replace(year=year_sat)

Timestamp('2014-02-01 00:28:48')

In [38]:
dd

Timestamp('2000-02-01 00:28:48')

In [40]:
for k in nmp.arange(len(date_model)):
    dd=date_model[k]
    print(k, dd.replace(year=year_sat))

0 2014-02-01 00:28:48
1 2014-02-01 01:29:36
2 2014-02-01 02:29:20
3 2014-02-01 03:29:04
4 2014-02-01 04:28:48
5 2014-02-01 05:29:36
6 2014-02-01 06:29:20
7 2014-02-01 07:29:04
8 2014-02-01 08:28:48
9 2014-02-01 09:29:36
10 2014-02-01 10:29:20
11 2014-02-01 11:29:04
12 2014-02-01 12:28:48
13 2014-02-01 13:29:36
14 2014-02-01 14:29:20
15 2014-02-01 15:29:04
16 2014-02-01 16:28:48
17 2014-02-01 17:29:36
18 2014-02-01 18:29:20
19 2014-02-01 19:29:04
20 2014-02-01 20:28:48
21 2014-02-01 21:29:36
22 2014-02-01 22:29:20
23 2014-02-01 23:29:04
24 2014-02-02 00:28:48
25 2014-02-02 01:29:36
26 2014-02-02 02:29:20
27 2014-02-02 03:29:04
28 2014-02-02 04:28:48
29 2014-02-02 05:29:36
30 2014-02-02 06:29:20
31 2014-02-02 07:29:04
32 2014-02-02 08:28:48
33 2014-02-02 09:29:36
34 2014-02-02 10:29:20
35 2014-02-02 11:29:04
36 2014-02-02 12:28:48
37 2014-02-02 13:29:36
38 2014-02-02 14:29:20
39 2014-02-02 15:29:04
40 2014-02-02 16:28:48
41 2014-02-02 17:29:36
42 2014-02-02 18:29:20
43 2014-02-02 19:29:0

ValueError: day is out of range for month

In [42]:
date_model[672]

Timestamp('2000-02-29 00:28:48')

In [43]:
dsmod

<xarray.Dataset>
Dimensions:     (eta_rho: 1489, eta_v: 1488, time: 2280, xi_rho: 1491, xi_u: 1490)
Dimensions without coordinates: eta_rho, eta_v, time, xi_rho, xi_u
Data variables: (12/13)
    angle       (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    f           (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    h           (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    lat         (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    lon         (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    ocean_time  (time) float32 dask.array<chunksize=(24,), meta=np.ndarray>
    ...          ...
    pn          (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    salt        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>
    temp        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>
    u           (time, eta_rho, xi_u) float32 dask.array<chunksize=(24, 1489, 1490), meta=np.ndarray>
    v           (time, eta_v, xi_rho) float32 dask.array<chunksize=(24, 1488, 1491), meta=np.ndarray>
    zeta        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>

In [44]:
vtime=dsmod.ocean_time

In [51]:
from datetime import datetime as dtm
date=[]
for k in nmp.arange(len(vtime)):
    date.append(nmp.array(dtm.utcfromtimestamp(vtime[k]), dtype='datetime64')) 
date=nmp.array(date)


In [57]:
print(type(date))

<class 'numpy.ndarray'>


In [56]:
rt1=nmp.array('2000-02-29T00:00:00')
print(type(rt1))

<class 'numpy.ndarray'>


In [ ]:
idx1=nmp.where(date>rt1)


In [ ]:
idx2=nmp.where(date>rt)

In [59]:
from gonzag.io import GetTimeVector

In [61]:
rvt = GetTimeVector( dsmod, name_time_mod, lquiet=True )

In [63]:
(rtu1,rtu2)=nmp.array(['2000-02-29T00:00:00','2000-03-01T00:00:00'], dtype='datetime64')

In [64]:
jt1, jt2 = scan_idx( rvt, rtu1, rtu2 )

In [65]:
print(jt1,jt2)

744 767


In [68]:
rvt[jt1-1:jt2+2]

array(['2000-02-28T23:29:04.000000', '2000-02-29T00:28:48.000000',
       '2000-02-29T01:29:36.000000', '2000-02-29T02:29:20.000000',
       '2000-02-29T03:29:04.000000', '2000-02-29T04:28:48.000000',
       '2000-02-29T05:29:36.000000', '2000-02-29T06:29:20.000000',
       '2000-02-29T07:29:04.000000', '2000-02-29T08:28:48.000000',
       '2000-02-29T09:29:36.000000', '2000-02-29T10:29:20.000000',
       '2000-02-29T11:29:04.000000', '2000-02-29T12:28:48.000000',
       '2000-02-29T13:29:36.000000', '2000-02-29T14:29:20.000000',
       '2000-02-29T15:29:04.000000', '2000-02-29T16:28:48.000000',
       '2000-02-29T17:29:36.000000', '2000-02-29T18:29:20.000000',
       '2000-02-29T19:29:04.000000', '2000-02-29T20:28:48.000000',
       '2000-02-29T21:29:36.000000', '2000-02-29T22:29:20.000000',
       '2000-02-29T23:29:04.000000', '2000-03-01T00:28:48.000000'],
      dtype='datetime64[us]')

In [78]:
dsmod2=dsmod.drop_isel(time=nmp.arange(jt1,jt2+1,1))

In [79]:
rvt = GetTimeVector( dsmod2, name_time_mod, lquiet=True )

In [80]:
print(rvt[700:800])

['2000-02-27T04:28:48.000000' '2000-02-27T05:29:36.000000'
 '2000-02-27T06:29:20.000000' '2000-02-27T07:29:04.000000'
 '2000-02-27T08:28:48.000000' '2000-02-27T09:29:36.000000'
 '2000-02-27T10:29:20.000000' '2000-02-27T11:29:04.000000'
 '2000-02-27T12:28:48.000000' '2000-02-27T13:29:36.000000'
 '2000-02-27T14:29:20.000000' '2000-02-27T15:29:04.000000'
 '2000-02-27T16:28:48.000000' '2000-02-27T17:29:36.000000'
 '2000-02-27T18:29:20.000000' '2000-02-27T19:29:04.000000'
 '2000-02-27T20:28:48.000000' '2000-02-27T21:29:36.000000'
 '2000-02-27T22:29:20.000000' '2000-02-27T23:29:04.000000'
 '2000-02-28T00:28:48.000000' '2000-02-28T01:29:36.000000'
 '2000-02-28T02:29:20.000000' '2000-02-28T03:29:04.000000'
 '2000-02-28T04:28:48.000000' '2000-02-28T05:29:36.000000'
 '2000-02-28T06:29:20.000000' '2000-02-28T07:29:04.000000'
 '2000-02-28T08:28:48.000000' '2000-02-28T09:29:36.000000'
 '2000-02-28T10:29:20.000000' '2000-02-28T11:29:04.000000'
 '2000-02-28T12:28:48.000000' '2000-02-28T13:29:36.00000